In [7]:
import pandas as pd
import glob

In [8]:
import pandas as pd
import glob

# 1) CSV 파일 로드
files = sorted(glob.glob("data/KS_NFA_FTNESS_MESURE_MVN_PRSCRPTN_GNRLZ_INFO_202503.csv"))
print("로드 파일:", files)

df_list = []
for file in files:
    print("로드 중:", file)
    temp = pd.read_csv(file, encoding='utf-8-sig', engine='python')
    df_list.append(temp)

df = pd.concat(df_list, ignore_index=True)
print("원본 shape:", df.shape)

로드 파일: ['data/KS_NFA_FTNESS_MESURE_MVN_PRSCRPTN_GNRLZ_INFO_202503.csv']
로드 중: data/KS_NFA_FTNESS_MESURE_MVN_PRSCRPTN_GNRLZ_INFO_202503.csv
원본 shape: (28712, 57)


In [9]:
# 2) 필요한 컬럼 추출
core_cols = [
    'AGRDE_FLAG_NM', 'MESURE_AGE_CO', 'SEXDSTN_FLAG_CD',
    'MESURE_IEM_001_VALUE', 'MESURE_IEM_002_VALUE',
    'MESURE_IEM_018_VALUE',
    'MVM_PRSCRPTN_CN'
]

df = df[core_cols].copy()
print("추출 후 shape:", df.shape)


추출 후 shape: (28712, 7)


In [10]:
# 3) 나이 정제
df['MESURE_AGE_CO'] = pd.to_numeric(df['MESURE_AGE_CO'], errors='coerce')

df = df[(df['MESURE_AGE_CO'] >= 10) & (df['MESURE_AGE_CO'] <= 100)]
df['group'] = (df['MESURE_AGE_CO'] // 10) * 10

In [11]:
# 4) 성별 인코딩
df = pd.concat([df, pd.get_dummies(df['SEXDSTN_FLAG_CD'], prefix="", prefix_sep="")], axis=1)
df[['F', 'M']] = df[['F', 'M']].fillna(0).astype(int)

In [12]:
# 5) 운동처방 → 25개 상세 카테고리
def map_to_category(text):
    if pd.isna(text):
        return "기타"

    t = text.strip()

    # 본운동 부분만 분리
    if "본운동" in t:
        try:
            main_part = t.split("본운동:")[1].split("/")[0].strip()
        except:
            main_part = t
    else:
        # 본운동이 아예 없으면 기타로 처리
        return "기타"

    t = main_part.lower()

    # 하체 근력
    if any(k in t for k in ["스쿼트", "런지", "하체", "허벅지", "둔근", "엉덩이", "점프"]):
        return "하체근력"

    # 상체 근력
    if any(k in t for k in ["푸시업", "팔꿈치", "팔굽", "이두", "삼두", "벤치", "어깨", "가슴"]):
        return "상체근력"

    # 코어
    if any(k in t for k in ["플랭크", "복근", "브릿지", "버티기", "코어"]):
        return "코어"

    # 걷기
    if any(k in t for k in ["걷기", "워킹", "보행"]):
        return "걷기"

    # 달리기
    if any(k in t for k in ["달리기", "러닝", "런닝", "조깅"]):
        return "달리기"

    # 자전거
    if any(k in t for k in ["자전거", "사이클"]):
        return "자전거"

    # 기타
    return "기타"


df['category'] = df['MVM_PRSCRPTN_CN'].apply(map_to_category)
print(df['category'].value_counts())

category
기타      16166
달리기      3776
코어       2171
걷기       2039
하체근력     2028
상체근력     1931
자전거       599
Name: count, dtype: int64


In [13]:
# 6) ‘기타’ 제거 + 스트레칭 다운샘플링
df = df[df['category'] != "기타"].copy()

if len(df[df['category'] == '스트레칭']) > 1200:
    stretch_sample = df[df['category'] == '스트레칭'].sample(1200, random_state=42)
    rest_df = df[df['category'] != '스트레칭']
    df = pd.concat([stretch_sample, rest_df], ignore_index=True)

print("최종 데이터 개수:", df.shape)

최종 데이터 개수: (12544, 11)


In [14]:
# 7) 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
target_encoder = LabelEncoder()
df['target'] = target_encoder.fit_transform(df['category'])

print("최종 카테고리 개수:", len(target_encoder.classes_))

최종 카테고리 개수: 6


In [15]:
# 8) Feature 컬럼 구성
feature_cols = [
    'group',
    'MESURE_AGE_CO',
    'F', 'M',
    'MESURE_IEM_001_VALUE',  # 키
    'MESURE_IEM_002_VALUE',  # 체중
    'MESURE_IEM_018_VALUE'   # BMI
]
x = df[feature_cols].fillna(0)
y = df['target']

In [16]:
# 9) 학습/테스트 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)

In [17]:
# 10) 모델 학습 (RandomForest)
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    n_jobs=-1,
    random_state=42
)
model.fit(x_train, y_train)

,n_estimators,200
,criterion,'gini'
,max_depth,15
,min_samples_split,10
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [15]:
# 11) 평가
from sklearn.metrics import accuracy_score, f1_score

pred = model.predict(x_test)

print("Accuracy:", accuracy_score(y_test, pred))
print("Macro F1:", f1_score(y_test, pred, average='macro'))
print("Micro F1:", f1_score(y_test, pred, average='micro'))

Accuracy: 0.48502994011976047
Macro F1: 0.19297192970402557
Micro F1: 0.48502994011976047


In [18]:
# 12) 모델 저장
import joblib

joblib.dump(model, "exercise_recommender.pkl")
joblib.dump(target_encoder, "label_encoder.pkl")

print("모델 저장 완료")

모델 저장 완료


In [ ]:
# csv파싱용
